![alt text](https://moodle.msengineering.ch/pluginfile.php/1/core_admin/logo/0x150/1643104191/logo-mse.png "MSE Logo") 
![alt text](https://www.hes-so.ch/typo3conf/ext/wng_site/Resources/Public/HES-SO/img/logo_hesso_master_tablet.svg "Hes Logo")

# WEB MINING PROJET AIRBNB
## L'objectif
Dans ce notebook, nous avons créé des tableaux d'ANOVA pour identifier les facteurs significatifs influençant le prix et new rating. Ensuite, nous avons élaboré des modèles de régression, tels que la régression linéaire, la forêt aléatoire (Random Forest) et XGBoost, pour prédire les prix. Enfin, nous avons réalisé une analyse de clustering pour étudier la réduction des prix par région.

In [74]:
import pandas as pd
data = pd.read_csv("C:/Users/Abdi/Desktop/webmin_aout24/data/total_out_clean.csv")

### Table d'ANOVA pour le prix

In [48]:
# la formule pour le modèle ANOVA
formula = 'price ~ old_price + C(Type) + C(pays) + C(region) + voyageurs + rooms + bed + bathroom'
model = ols(formula, data=data).fit()

# Nous effectuons la calcule de table d'anova
anova_table = sm.stats.anova_lm(model, typ=2)
print("Table d'ANOVA:")
print(anova_table)

# Interpretation des effets des facteurs
print("\nInterprétation des effets des facteurs:")
for factor in anova_table.index:
    print(f"\nFacteur: {factor}")
    print(f"  - Sum of Squares: {anova_table.loc[factor, 'sum_sq']:.2f}")
    print(f"  - Degrees of Freedom (df): {anova_table.loc[factor, 'df']}")
    print(f"  - F-statistic: {anova_table.loc[factor, 'F']:.2f}")
    print(f"  - p-value: {anova_table.loc[factor, 'PR(>F)']:.4f}")
    if anova_table.loc[factor, 'PR(>F)'] < 0.05:
        print("  → Ce facteur a un effet statistiquement significatif sur le prix")
    else:
        print("  → Ce facteur n'a pas d'effet statistiquement significatif sur le prix")


Table d'ANOVA:
                 sum_sq     df           F        PR(>F)
C(Type)    9.012353e+06   30.0    1.663152  2.057488e-02
C(pays)    1.030455e+07    7.0    8.149786  7.245144e-09
C(region)  1.067696e+08  501.0    1.179845  7.492758e-02
old_price  7.342361e+07    1.0  406.490858  7.884314e-53
voyageurs  1.036661e+04    1.0    0.057392  8.108788e-01
rooms      1.153301e+05    1.0    0.638496  4.250769e-01
bed        2.745744e+04    1.0    0.152011  6.969801e-01
bathroom   6.312329e+05    1.0    3.494658  6.283240e-02
Residual   4.172506e+07  231.0         NaN           NaN

Interprétation des effets des facteurs:

Facteur: C(Type)
  - Sum of Squares: 9012353.46
  - Degrees of Freedom (df): 30.0
  - F-statistic: 1.66
  - p-value: 0.0206
  → Ce facteur a un effet statistiquement significatif sur le prix

Facteur: C(pays)
  - Sum of Squares: 10304554.53
  - Degrees of Freedom (df): 7.0
  - F-statistic: 8.15
  - p-value: 0.0000
  → Ce facteur a un effet statistiquement significatif su

### Coefficients du modèle pour le prix


In [50]:
# la formule pour le modèle de régression linéaire
formula = 'price ~ old_price + C(Type) + C(pays) + C(region) + voyageurs + rooms + bed + bathroom'
model = ols(formula, data=data).fit()
print("Coefficients du modèle:")
print(model.params)

# Interpréteration des coefficients
print("\nInterprétation des coefficients:")
for factor, coef in model.params.items():
    if coef > 0:
        direction = "positif"
    else:
        direction = "négatif"
    
    print(f"{factor}: effet {direction} ({coef:.4f}) sur le prix")


Coefficients du modèle:
Intercept                              -393.647095
C(Type)[T.Camping]                      -50.314146
C(Type)[T.Chambre dans appart'hôtel]    276.513563
C(Type)[T.Chambre dans hôtel]          -332.788520
C(Type)[T.Château]                      360.946465
                                           ...    
old_price                                 0.548718
voyageurs                                10.187153
rooms                                    91.344149
bed                                     -19.949506
bathroom                                188.040643
Length: 544, dtype: float64

Interprétation des coefficients:
Intercept: effet négatif (-393.6471) sur le prix
C(Type)[T.Camping]: effet négatif (-50.3141) sur le prix
C(Type)[T.Chambre dans appart'hôtel]: effet positif (276.5136) sur le prix
C(Type)[T.Chambre dans hôtel]: effet négatif (-332.7885) sur le prix
C(Type)[T.Château]: effet positif (360.9465) sur le prix
C(Type)[T.Conteneur industriel]: effet positif

In [20]:
data.head()

,Unnamed: 0,price,old_price,voyageurs,rooms,bed,bathroom,period_start,period_end,new_rating,...,region_Wolfertswil,region_Wolfhagen,region_Wundschuh,region_Yutz,region_Zehethof,region_Zirkitzen,region_Zürich,region_car/caravane - Carrazeda de Ansiães,region_car/caravane - Francfort-sur-l'Oder,region_Érezée
0,0,2920.0,6084.0,9,3,8,2,2024-07-31,2024-08-31,4.97,...,False,False,False,False,False,False,False,False,False,False
1,1,1500.0,1705.0,2,1,1,1,2024-07-31,2024-08-31,4.89,...,False,False,False,False,False,False,False,False,False,False
2,2,1968.0,3280.0,2,1,1,1,2024-07-31,2024-08-31,4.81,...,False,False,False,False,False,False,False,False,False,False
3,3,2353.0,2870.0,4,2,3,1,2024-07-31,2024-08-31,4.96,...,False,False,False,False,False,False,False,False,False,False
4,4,2129.0,2505.0,2,1,1,1,2024-07-31,2024-08-31,4.83,...,False,False,False,False,False,False,False,False,False,False


### Table d'anova pour New rating

In [27]:
# Nous convertions les variables catégorielles en type 'category'
data['Type'] = data['Type'].astype('category')
data['region'] = data['region'].astype('category')

# La formule pour l'ANOVA et l'ajustement du modèle
formula = 'new_rating ~ price + old_price + C(Type) + C(region) + voyageurs + rooms + bed + bathroom'
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# les facteurs significatifs (p-value < 0.05)
significant_factors = anova_table[anova_table['PR(>F)'] < 0.05]

# Affichage des coefficients du modèle ajusté
coefficients = model.params

# Nous filtrons les coefficients pour montrer les effets des facteurs significatifs
significant_effects = pd.concat([
    coefficients.filter(like='Type'),
    coefficients.filter(like='region'),
    coefficients.loc[['price', 'rooms']]
])

print("Effets des facteurs significatifs sur new_rating:")
print(significant_effects)


Effets des facteurs significatifs sur new_rating:
C(Type)[T.Camping]                                 -0.425468
C(Type)[T.Chambre dans appart'hôtel]               -0.121275
C(Type)[T.Chambre dans hôtel]                       0.124513
C(Type)[T.Château]                                 -0.049836
C(Type)[T.Conteneur industriel]                     0.661877
                                                      ...   
C(region)[T.car/caravane - Carrazeda de Ansiães]   -0.939249
C(region)[T.car/caravane - Francfort-sur-l'Oder]    0.513780
C(region)[T.Érezée]                                 0.060185
price                                               0.000029
rooms                                              -0.099890
Length: 533, dtype: float64


In [51]:
# changement(convertir) des variables catégorielles en type category
data['Type'] = data['Type'].astype('category')
data['region'] = data['region'].astype('category')

# la formule pour l'ANOVA
formula = 'new_rating ~ price + old_price + C(Type) + C(region) + voyageurs + rooms + bed + bathroom'
model = ols(formula, data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("Tableau ANOVA complet:")
print(anova_table)
significant_factors = anova_table[anova_table['PR(>F)'] < 0.05]
print("\nFacteurs significatifs (p-value < 0.05):")
print(significant_factors)


Tableau ANOVA complet:
              sum_sq     df         F        PR(>F)
C(Type)     2.674592   30.0  3.288400  1.905981e-07
C(region)  16.513900  501.0  1.215795  4.452229e-02
price       0.036308    1.0  1.339225  2.483659e-01
old_price   0.013171    1.0  0.485808  4.865042e-01
voyageurs   0.020751    1.0  0.765399  3.825530e-01
rooms       0.140894    1.0  5.196851  2.354061e-02
bed         0.051443    1.0  1.897483  1.696935e-01
bathroom    0.000074    1.0  0.002727  9.583991e-01
Residual    6.262728  231.0       NaN           NaN

Facteurs significatifs (p-value < 0.05):
              sum_sq     df         F        PR(>F)
C(Type)     2.674592   30.0  3.288400  1.905981e-07
C(region)  16.513900  501.0  1.215795  4.452229e-02
rooms       0.140894    1.0  5.196851  2.354061e-02


### Comparaison des modèles de régression pour la prédiction des prix

In [58]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

#  Conversion des variables catégorielles en variables numériques à l'aide du codage one-hot
data = pd.get_dummies(data, columns=['Type', 'pays', 'region'], drop_first=True)

# features : (X) et  la variable cible (y) 
X = data[['old_price', 'rooms', 'bed', 'bathroom'] + 
         [col for col in data.columns if 'Type_' in col or 'pays_' in col or 'region_' in col]]
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model 1: Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

# Model 2: Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Model 3: XGBoost Regression
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgboost_model.fit(X_train, y_train)
y_pred_xgb = xgboost_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Comparaison des modèles
comparison = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest", "XGBoost"],
    "Mean Squared Error": [mse_lr, mse_rf, mse_xgb],
    "R^2": [r2_lr, r2_rf, r2_xgb]
})

print("Model Comparison")
print(comparison)



Model Comparison
               Model  Mean Squared Error       R^2
0  Linear Regression       353866.633647  0.555967
1      Random Forest       197681.458723  0.751949
2            XGBoost       209173.565960  0.737528


In [15]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Création des génériques données pour prédiction des prix en utilisant RF modèle

In [67]:
generic_data = pd.DataFrame({
    'old_price': [2000, 3000, 4000],  
    'rooms': [2, 3, 4],              
    'bed': [1, 2, 3],                 
    'bathroom': [1, 2, 2],            
    'Type_apartment': [1, 0, 0],      
    'Type_loft': [0, 1, 0],          
    'Type_cottage': [0, 0, 1],        
    'pays_Spain': [1, 0, 1],          
    'region_Madrid': [1, 0, 0],      
    'region_Seville': [0, 1, 0],     
    
})

for col in X_train.columns:
    if col not in generic_data.columns:
        generic_data[col] = 0

generic_data = generic_data[X_train.columns]
predicted_prices = rf_model.predict(generic_data)
generic_data['predicted_price'] = predicted_prices
print(generic_data[['old_price', 'rooms', 'bed', 'bathroom', 'predicted_price']])


C:\Users\Abdi\AppData\Local\Temp\ipykernel_2528\4171585614.py:26: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Abdi\AppData\Local\Temp\ipykernel_2528\4171585614.py:26: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Abdi\AppData\Local\Temp\ipykernel_2528\4171585614.py:26: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

   old_price  rooms  bed  bathroom  predicted_price
0       2000      2    1         1          1430.78
1       3000      3    2         2          1846.71
2       4000      4    3         2          2699.28


C:\Users\Abdi\AppData\Local\Temp\ipykernel_2528\4171585614.py:35: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



### Visualisation de prediction de prix de Random Forest modèle


In [72]:
import plotly.express as px
# Predicted vs Actual Prices
result_df = pd.DataFrame({'Actual Price': y_test, 'Predicted Price': y_pred_rf})
fig2 = px.scatter(result_df, x='Actual Price', y='Predicted Price', 
                  title='Random Forest: Predicted vs Actual Prices',
                  labels={'Actual Price': 'Actual Price', 'Predicted Price': 'Predicted Price'},
                  trendline='ols')

fig2.show()


In [75]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
from statsmodels.formula.api import ols

# réduction de prix 
data['price_reduction'] = data['old_price'] - data['price']
# réduction moyenne de prix par région
region_reduction = data.groupby('region')['price_reduction'].mean().reset_index()
# Normalisation des données de réduction de prix
scaler = StandardScaler()
region_reduction_normalized = scaler.fit_transform(region_reduction[['price_reduction']])
# clustering K-Means
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=0)
region_clusters = kmeans.fit_predict(region_reduction_normalized)
# Ajouter les informations de cluster au DataFrame
region_reduction['cluster'] = region_clusters
# Création d'un graphique interactif avec bleu, rouge, et vert pour les clusters
fig = px.scatter(region_reduction, 
                 x=region_reduction.index, 
                 y='price_reduction', 
                 color=region_reduction['cluster'].astype(str),  
                 title='Clustering des Regions par Réduction de prix',
                 labels={'x': 'Region Index', 'price_reduction': 'Price Reduction'},
                 hover_data=['region'],
                 color_discrete_map={
                     '0': 'blue',
                     '1': 'green',
                     '2': 'red'
                 })

fig.update_layout(showlegend=True, xaxis_title='Region Index', yaxis_title='Price Reduction')
fig.show()
